# Carga Gold - Fato Carteira

Este notebook realiza a carga da fato de cotação (fato_cotacao) a partir dos dados da tabela staging de cotação histórica.

## Imports

In [1]:
from delta import configure_spark_with_delta_pip
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.window import Window

## Start Spark Session

In [ ]:
import os
from pyspark.sql import SparkSession

# ===== CONFIGURAÇÃO DE DIRETÓRIOS =====
BASE_DIR = "D:/Projetos/DataLake"
WAREHOUSE_DIR = f"{BASE_DIR}/spark-warehouse"
METASTORE_DIR = f"{BASE_DIR}/metastore_db"
SCRATCH_DIR = f"{BASE_DIR}/hive_scratch"

# Criar diretórios necessários
for dir_path in [WAREHOUSE_DIR, SCRATCH_DIR]:
    os.makedirs(dir_path, exist_ok=True)

# ===== CONFIGURAÇÃO DO HADOOP (Windows) =====
# Se você tem o winutils instalado, descomente e ajuste o caminho:
# os.environ['HADOOP_HOME'] = r'C:\hadoop'

# ===== SPARK SESSION =====
builder = (
    SparkSession.builder
    .appName("Carga fato carteira")
    .master("local[*]")
    # Delta Lake
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    # Hive Local (persistente)
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.warehouse.dir", f"file:///{WAREHOUSE_DIR}")
    .config("hive.metastore.warehouse.dir", f"file:///{WAREHOUSE_DIR}")
    .config(
        "javax.jdo.option.ConnectionURL",
        f"jdbc:derby:;databaseName={METASTORE_DIR};create=true"
    )
    # Corrige problemas no Windows
    .config("hive.exec.scratchdir", SCRATCH_DIR)
    .config("hive.metastore.schema.verification", "false")
    .config("hive.metastore.schema.verification.record.version", "false")
    .config("datanucleus.schema.autoCreateAll", "true")
    .enableHiveSupport()
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

print(f"\n✅ Spark {spark.version} iniciado com Hive local persistente!")
print(f"📁 Warehouse: {WAREHOUSE_DIR}")
print(f"📁 Metastore: {METASTORE_DIR}\n")



✅ Spark 3.5.7 iniciado com Hive local persistente!



## Define Delta Table Paths

In [3]:
# Define caminhos locais onde serão armazenadas as tabelas Delta
base_silver_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/silver"
base_gold_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/gold"

# Define caminhos das tabelas Delta
delta_path_controle_ativo = f"{base_silver_path}/stg_controle_ativo"
delta_path_dim_ativo = f"{base_gold_path}/dim_ativo_financeiro"
delta_path_fato_carteira = f"{base_gold_path}/fato_carteira"

In [4]:
"""
Pipeline para carregar posição mensal de carteira de investimentos
Abordagem funcional
"""
def extrair_operacoes_mes(spark: SparkSession, mes_ref: str, cpf: str = None) -> DataFrame:
    """
    Extrai operações do mês da tabela silver
    
    Args:
        spark: SparkSession
        mes_ref: Mês de referência no formato 'YYYY-MM'
        cpf: CPF do investidor (opcional)
    """
    query = f"""
    SELECT 
        mes_ref,
        cpf,
        cotista,
        cd_ativo,
        dt_operacao,
        cd_tipo_operacao,
        qt_operacao,
        vl_preco_ativo,
        vl_custo_total,
        vl_liquido,
        qt_estoque,
        vl_pmedio,
        vl_ganho_perda,
        ir_mes,
        vl_vendas_mes,
        ts_insercao
    FROM silver.stg_controle_ativo
    WHERE mes_ref = '{mes_ref}'
    """
    
    if cpf:
        query += f" AND cpf = '{cpf}'"
        
    return spark.sql(query)


def calcular_posicao_final(df_operacoes: DataFrame) -> DataFrame:
    """
    Calcula a posição final de cada ativo no mês
    Pega a última operação de cada ativo
    """
    window_spec = Window.partitionBy("cpf", "cd_ativo", "mes_ref") \
                        .orderBy(F.desc("dt_operacao"), F.desc("ts_insercao"))
    
    return df_operacoes \
        .withColumn("rank", F.row_number().over(window_spec)) \
        .filter(F.col("rank") == 1) \
        .select(
            "mes_ref",
            "cpf",
            "cotista",
            "cd_ativo",
            "qt_estoque",
            "vl_pmedio",
            F.expr("qt_estoque * vl_pmedio").alias("vl_posicao"),
            "vl_ganho_perda",
            "ir_mes",
            "vl_vendas_mes"
        )


def agregar_por_cotista(df_posicao: DataFrame) -> DataFrame:
    """
    Agrega métricas por cotista
    """
    return df_posicao.groupBy("mes_ref", "cpf", "cotista").agg(
        F.count("cd_ativo").alias("qt_ativos_carteira"),
        F.sum("qt_estoque").alias("qt_total_acoes"),
        F.sum("vl_posicao").alias("vl_total_carteira"),
        F.sum("vl_ganho_perda").alias("vl_total_ganho_perda"),
        F.sum("ir_mes").alias("vl_total_ir_mes"),
        F.sum("vl_vendas_mes").alias("vl_total_vendas_mes"),
        F.avg("vl_pmedio").alias("vl_pmedio_ponderado")
    )


def calcular_metricas_financeiras(df_agregado: DataFrame) -> DataFrame:
    """
    Adiciona métricas financeiras calculadas
    """
    return df_agregado \
        .withColumn(
            "pc_rentabilidade_mes",
            F.when(F.col("vl_total_carteira") > 0,
                   (F.col("vl_total_ganho_perda") / F.col("vl_total_carteira")) * 100
            ).otherwise(0)
        ) \
        .withColumn(
            "pc_ir_sobre_ganho",
            F.when(F.col("vl_total_ganho_perda") > 0,
                   (F.col("vl_total_ir_mes") / F.col("vl_total_ganho_perda")) * 100
            ).otherwise(0)
        ) \
        .withColumn(
            "vl_liquido_apos_ir",
            F.col("vl_total_ganho_perda") - F.col("vl_total_ir_mes")
        ) \
        .withColumn(
            "ts_processamento",
            F.current_timestamp()
        )


def salvar_posicao_mensal(df_gold: DataFrame, modo: str = "append"):
    """
    Salva posição mensal agregada na camada gold
    """
    df_gold.write \
        .format("delta") \
        .mode(modo) \
        .option("mergeSchema", "true") \
        .saveAsTable("gold.posicao_mensal_carteira")
    
    print(f"✓ Posição mensal salva em gold.posicao_mensal_carteira")


def salvar_posicao_detalhada(df_posicao: DataFrame, modo: str = "append"):
    """
    Salva posição detalhada por ativo na camada gold
    """
    df_detalhada = df_posicao.withColumn("ts_processamento", F.current_timestamp())
    
    df_detalhada.write \
        .format("delta") \
        .mode(modo) \
        .option("mergeSchema", "true") \
        .saveAsTable("gold.posicao_detalhada_ativo")
    
    print(f"✓ Posição detalhada salva em gold.posicao_detalhada_ativo")



In [5]:
 # 1. Extrair operações
mes_ref = "2024-05"
cpf = None  # Ou defina um CPF específico para filtrar

print("1. Extraindo operações da silver...")
df_operacoes = extrair_operacoes_mes(spark, mes_ref, cpf)
count_operacoes = df_operacoes.count()
print(f"   → {count_operacoes} operações encontradas")

df_operacoes.show(5)

1. Extraindo operações da silver...


AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient

## Read Source Data

In [ ]:
# Lê a tabela staging de cotação histórica
df_stg_controle_ativo = spark.read.format("delta").load(delta_path_controle_ativo)

print(f"[SUCESSO] Leitura da tabela staging em: {delta_path_controle_ativo}")
print(f"Total de registros: {df_stg_controle_ativo.count()}")

[SUCESSO] Leitura da tabela staging em: D:/Projetos/Jornada_financas_pessoais/data/delta/silver/stg_cotacao_historica
Total de registros: 5126223
[SUCESSO] Leitura da tabela dimensão em: D:/Projetos/Jornada_financas_pessoais/data/delta/gold/dim_ativo_financeiro
Total de registros: 2262


## Transform Data

In [ ]:
# Filtra apenas registros com tp_mercado = '10'
df_stg_cotacao_historica = df_stg_cotacao_historica.filter(col("tp_mercado") == "010")

# Join LEFT (mantém todas as cotações mesmo sem correspondência na dimensão)
df_joined = (
    df_stg_cotacao_historica.alias("stg")
    .join(
        df_dim_ativo.alias("dim"),
        col("stg.cd_negociacao") == col("dim.cd_ativo"),
        "left"  # mantém as linhas da stg mesmo se não achar na dimensão
    )
)

# Tratamento da chave surrogate faltante (usa -1)
df_fato_cotacao = (
    df_joined.select(
        col("stg.dt_pregao"),
        when(col("sk_ativo").isNull(), lit("-1")).otherwise(col("sk_ativo")).alias("sk_ativo"),
        col("vl_abertura"),
        col("vl_minimo"),
        col("vl_maximo"),
        col("vl_medio"),
        col("vl_ultimo_negocio"),
        col("qt_negocios_efetuados").alias("qt_negocio"),
        col("qt_total_titulos").alias("qt_titulo"),
        col("vl_total_titulos").alias("vl_volume"),
        current_timestamp().alias("ts_insercao"),
        year(col("dt_pregao")).alias("ano_pregao"),
        month(col("dt_pregao")).alias("mes_pregao")
    )
)

## Write Data

In [ ]:

df_fato_cotacao.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "false") \
    .option("partitionOverwriteMode", "dynamic") \
    .partitionBy("ano_pregao", "mes_pregao") \
    .save(delta_path_fato_cotacao)

print("✅ Dynamic partition overwrite executado - apenas partições afetadas foram sobrescritas")

✅ Dynamic partition overwrite executado - apenas partições afetadas foram sobrescritas


## Stop Spark Session

In [ ]:
# Encerra a SparkSession
spark.stop()